## 2.1. Problem Statement: Machine Learning 7KNN
In this assignment, students will be using the K-nearest neighbors
algorithm to predict how many points NBA players scored in the 2013-2014
season.
A look at the data
Before we dive into the algorithm, let’s take a look at our data. Each row in
the data contains information on how a player performed in the 2013-2014
NBA season.
Download 'nba_2013.csv' file from this link:
https://www.dropbox.com/s/b3nv38jjo5dxcl6/nba_2013.csv?dl=0
Here are some selected columns from the data:
player - name of the player
pos - the position of the player
g - number of games the player was in
gs - number of games the player started
pts - total points the player scored
There are many more columns in the data, mostly containing information
about average player game performance over the course of the season.
See this site for an explanation of the rest of them.
We can read our dataset in and figure out which columns are present:
import pandas
with open("nba_2013.csv", 'r') as csvfile:
nba = pandas.read_csv(csvfile)

### Loading All the required Packages

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [46]:
# Loading the packages and reading the nba_dataset
import pandas as pd
#with open("nba_2013.csv", 'r') as csvfile:
nba = pd.read_csv("C:\\Users\\vmjoseph\\Desktop\\Python Training and Codes\\Acadgild\\Assignments\\ML\\nba_2013.csv")
print(nba.columns.values) # The names of all the columns in the data.

['player' 'pos' 'age' 'bref_team_id' 'g' 'gs' 'mp' 'fg' 'fga' 'fg.' 'x3p'
 'x3pa' 'x3p.' 'x2p' 'x2pa' 'x2p.' 'efg.' 'ft' 'fta' 'ft.' 'orb' 'drb'
 'trb' 'ast' 'stl' 'blk' 'tov' 'pf' 'pts' 'season' 'season_end']


### EDA/ Required dependant value Selection and missing value treatment

In [47]:
# EDA on the nba dataset and the respective columns
nba.head()

,player,pos,age,bref_team_id,g,gs,mp,fg,fga,fg.,...,drb,trb,ast,stl,blk,tov,pf,pts,season,season_end
0,Quincy Acy,SF,23,TOT,63,0,847,66,141,0.468,...,144,216,28,23,26,30,122,171,2013-2014,2013
1,Steven Adams,C,20,OKC,81,20,1197,93,185,0.503,...,190,332,43,40,57,71,203,265,2013-2014,2013
2,Jeff Adrien,PF,27,TOT,53,12,961,143,275,0.520,...,204,306,38,24,36,39,108,362,2013-2014,2013
3,Arron Afflalo,SG,28,ORL,73,73,2552,464,1011,0.459,...,230,262,248,35,3,146,136,1330,2013-2014,2013
4,Alexis Ajinca,C,25,NOP,56,30,951,136,249,0.546,...,183,277,40,23,46,63,187,328,2013-2014,2013


In [44]:
## Check the shape of the dataset
nba.shape

(481, 31)

In [48]:
## Check if dataset has null values
nba.isnull().sum().sum()

94

In [49]:
nba.describe()

,age,g,gs,mp,fg,fga,fg.,x3p,x3pa,x3p.,...,orb,drb,trb,ast,stl,blk,tov,pf,pts,season_end
count,481.000000,481.000000,481.000000,481.000000,481.000000,481.000000,479.000000,481.000000,481.000000,414.000000,...,481.000000,481.000000,481.000000,481.000000,481.000000,481.000000,481.000000,481.000000,481.000000,481.0
mean,26.509356,53.253638,25.571726,1237.386694,192.881497,424.463617,0.436436,39.613306,110.130977,0.285111,...,55.810811,162.817048,218.627859,112.536383,39.280665,24.103950,71.862786,105.869023,516.582121,2013.0
std,4.198265,25.322711,29.658465,897.258840,171.832793,368.850833,0.098672,50.855639,132.751732,0.157633,...,62.101191,145.348116,200.356507,131.019557,34.783590,30.875381,62.701690,71.213627,470.422228,0.0
min,19.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2013.0
25%,23.000000,32.000000,0.000000,388.000000,47.000000,110.000000,0.400500,0.000000,3.000000,0.234355,...,12.000000,43.000000,55.000000,20.000000,9.000000,4.000000,21.000000,44.000000,115.000000,2013.0
50%,26.000000,61.000000,10.000000,1141.000000,146.000000,332.000000,0.438000,16.000000,48.000000,0.330976,...,35.000000,135.000000,168.000000,65.000000,32.000000,14.000000,58.000000,104.000000,401.000000,2013.0
75%,29.000000,76.000000,54.000000,2016.000000,307.000000,672.000000,0.479500,68.000000,193.000000,0.375000,...,73.000000,230.000000,310.000000,152.000000,60.000000,32.000000,108.000000,158.000000,821.000000,2013.0
max,39.000000,83.000000,82.000000,3122.000000,849.000000,1688.000000,1.000000,261.000000,615.000000,1.000000,...,440.000000,783.000000,1114.000000,721.000000,191.000000,219.000000,295.000000,273.000000,2593.000000,2013.0


In [3]:
nba.isnull().sum()

player           0
pos              0
age              0
bref_team_id     0
g                0
gs               0
mp               0
fg               0
fga              0
fg.              2
x3p              0
x3pa             0
x3p.            67
x2p              0
x2pa             0
x2p.             3
efg.             2
ft               0
fta              0
ft.             20
orb              0
drb              0
trb              0
ast              0
stl              0
blk              0
tov              0
pf               0
pts              0
season           0
season_end       0
dtype: int64

In [50]:
# Replace NaN values with mean values.
nba = nba.fillna(nba.mean())

In [51]:
# Select Lebron James from our dataset
selected_player = nba[nba["player"] == "LeBron James"].iloc[0]

In [52]:
# Choose only the numeric columns (we'll use these to compute euclidean distance)
distance_columns = ['age', 'g', 'gs', 'mp', 'fg', 'fga', 'fg.', 'x3p', 'x3pa', 'x3p.', 'x2p', 'x2pa', 'x2p.', 'efg.', 'ft', 'fta', 'ft.', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts']

### Approach 1 Manual calculation of Euclidean distance using Lebron James as reference

In [53]:
#Defining the Euclidean diatance calculation for k-means
import math as mt
def euclidean_distance(row):
    """
    A simple euclidean distance function
    """
    inner_value = 0
    for k in distance_columns:
        inner_value += (row[k] - selected_player[k]) ** 2
    return mt.sqrt(inner_value)

# Find the distance from each player in the dataset to lebron.
lebron_distance = nba.apply(euclidean_distance, axis=1)

In [54]:
lebron_distance

0      3475.792868
1      3148.394999
2      3161.567339
3      1189.554979
4      3216.773098
          ...     
476    1948.158130
477    1851.909840
478     949.668916
479    2699.963932
480    3075.753429
Length: 481, dtype: float64

### Euclidean Distance Standardization

In [11]:
# Select only the numeric columns from the NBA dataset
nba_numeric = nba[distance_columns]
# Normalize all of the numeric columns
nba_normalized = (nba_numeric - nba_numeric.mean()) / nba_numeric.std()

In [55]:
from scipy.spatial import distance

# Fill in NA values in nba_normalized
nba_normalized.fillna(nba_normalized.mean(), inplace=True)

# Find the normalized vector for lebron james.
lebron_normalized = nba_normalized[nba["player"] == "LeBron James"]

# Find the distance between lebron james and everyone else.
euclidean_distances = nba_normalized.apply(lambda row: distance.euclidean(row, lebron_normalized), axis=1)

In [56]:

# Create a new dataframe with distances.
distance_frame = pd.DataFrame(data={"dist": euclidean_distances, "idx": euclidean_distances.index})
distance_frame.sort_values("dist", inplace=True)
# Find the most similar player to lebron (the lowest distance to lebron is lebron, the second smallest is the most similar non-lebron player)
second_smallest = distance_frame.iloc[1]["idx"]
most_similar_to_lebron = nba.loc[int(second_smallest)]["player"]
distance_frame

,dist,idx
225,0.000000,225
17,4.100792,17
136,4.146676,136
128,4.368930,128
185,4.435485,185
...,...,...
219,18.449732,219
397,18.456207,397
351,18.456400,351
190,18.469063,190


### Test-Train Split for the data Set

In [57]:
import random
from numpy.random import permutation

# Randomly shuffle the index of nba.
random_indices = permutation(nba.index)

# Set a cutoff for how many items we want in the test set (in this case 1/3 of the items)
test_cutoff = mt.floor(len(nba)/3)

# Generate the test set by taking the first 1/3 of the randomly shuffled indices.
test = nba.loc[random_indices[1:test_cutoff]]

# Generate the train set with the rest of the data.
train = nba.loc[random_indices[test_cutoff:]]

### Applying KNN Algorithn for 5 nearest neighbours

In [61]:
# The columns that we will be making predictions with.
x_columns = ['age', 'g', 'gs', 'mp', 'fg', 'fga', 'x3p', 'x3pa', 'x3p.', 'x2p', 'x2pa', 'ft', 'fta', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf']
# The column that we want to predict.
y_column = ["pts"]

from sklearn.neighbors import KNeighborsRegressor
# Create the knn model.
# Look at the five closest neighbors.
knn = KNeighborsRegressor(n_neighbors=5)
# Fit the model on the training data.
knn.fit(train[x_columns], train[y_column])
# Make point predictions on the test set using the fit model.
predictions = knn.predict(test[x_columns])
predictions

array([[7.3760e+02],
       [8.1000e+02],
       [1.1600e+01],
       [7.9980e+02],
       [9.0920e+02],
       [4.6400e+02],
       [8.6580e+02],
       [1.0000e+00],
       [5.9380e+02],
       [2.4280e+02],
       [1.1822e+03],
       [4.8000e+00],
       [9.8560e+02],
       [4.1240e+02],
       [1.3472e+03],
       [9.0920e+02],
       [3.6120e+02],
       [8.3820e+02],
       [3.2600e+01],
       [1.9800e+01],
       [1.0000e+00],
       [3.1160e+02],
       [1.0000e+00],
       [8.4000e+00],
       [1.0774e+03],
       [4.2940e+02],
       [1.5094e+03],
       [8.6800e+01],
       [9.7320e+02],
       [9.7200e+02],
       [1.5000e+01],
       [1.7160e+02],
       [9.2400e+01],
       [3.1000e+02],
       [1.5694e+03],
       [2.4000e+00],
       [2.9400e+01],
       [1.0000e+00],
       [7.8660e+02],
       [1.0028e+03],
       [8.0200e+02],
       [2.4000e+00],
       [7.9120e+02],
       [5.0540e+02],
       [1.2144e+03],
       [1.0200e+01],
       [8.4300e+02],
       [1.898

### Model Evaluation using R2/MSE/MAE/MAPE

In [85]:
r2_score(test[y_column],predictions) 

0.9676775284092742

In [86]:
# Get the actual values for the test set.
actual = test[y_column]
# Compute the mean squared error of our predictions.
mse = (((actual-predictions) ** 2).sum()) / len(predictions)
mse


pts    7119.525786
dtype: float64

In [63]:
MAE=abs((actual- predictions)).sum()/len(predictions)
MAE

pts    50.132075
dtype: float64

In [80]:
MAPE=(abs((actual- predictions).sum()/actual.sum())*100/len(predictions))
MAPE

pts    0.011184
dtype: float64

### Approach 2 KNN Using Grid Search

In [81]:
knnmodel = KNeighborsRegressor()
knnmodel.fit(train[x_columns], train[y_column])

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

In [82]:
pred=knnmodel.predict(test[x_columns])

In [84]:
##validating with R square
r2_score(test[y_column],pred) 

0.9676775284092742

### Apply Hyper parameter Tuning to find the best parameters

In [90]:
#With Hyper Parameters Tuning
#importing modules
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
#making the instance
model = KNeighborsRegressor(n_jobs=-1)
#Hyper Parameters Set
params = {'n_neighbors':[1,3,5,7,9,11,13,15,17,19,21,23,25],
          'leaf_size':[1,2,3,5],
          'weights':['uniform', 'distance'],
          'n_jobs':[-1]}
#Making models with hyper parameters sets
model1 = GridSearchCV(model, param_grid=params, n_jobs=1)
#Learning
model1.fit(train[x_columns], train[y_column])
#The best hyper parameters set
print("Best Hyper Parameters:\n",model1.best_params_)
#Prediction
prediction=model1.predict(test[x_columns])
#importing the metrics module
from sklearn import metrics
#evaluation(Accuracy)
print("Accuracy:",r2_score(test[y_column],pred) )

E:\Anaconda4\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best Hyper Parameters:
 {'leaf_size': 1, 'n_jobs': -1, 'n_neighbors': 5, 'weights': 'distance'}
Accuracy: 0.9676775284092742


### Model Evaluation using R2/MSE/MAE


In [91]:
print('The Mean Sqaured Error is : ', mean_squared_error(test[y_column],pred))
print('The Mean Absolute Error is : ', mean_absolute_error(test[y_column],pred))
print('The Model accuracy/R2 Score using K Nearest Neighbor Algorithm is ', r2_score(test[y_column],pred))

The Mean Sqaured Error is :  7119.525786163523
The Mean Absolute Error is :  50.132075471698116
The Model accuracy/R2 Score using K Nearest Neighbor Algorithm is  0.9676775284092742


In [ ]:
#def mean_absolute_percentage_error(y_true, y_pred): 
#   y_true, y_pred = np.array(y_true), np.array(y_pred)
#   return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


In [ ]:
#-------------------------------------------------------------#